In [87]:
from convokit import Corpus, download
corpus = Corpus(filename=download("wikipedia-politeness-corpus"))

Dataset already exists at C:\Users\Yuna\.convokit\downloads\wikipedia-politeness-corpus


In [88]:
import pandas as pd
import numpy as np
utt_txt = []
politeness = []
for utt in corpus.iter_utterances():
    utt_txt.append(utt.text)
    politeness.append(utt.meta['Binary'])

In [89]:
import re
questions = []
txt = []
patt = '(.+[.?!:])(.+\?)'
for utt in utt_txt:
    t = re.split(patt, utt)
    if len(t)==4:
        txt.append(t[1])
        questions.append(t[2])
    else:
        txt.append(np.nan)
        questions.append(np.nan)


In [90]:
df = pd.DataFrame({'prompt':txt, 'completion': questions, 'politeness':politeness})
df = df.dropna()
df.head()

,prompt,completion,politeness
0,Where did you learn English?,How come you're taking on a third language?,-1
1,Thanks very much for your edit to the <url> ar...,Would you be interested in tackling the <url>...,1
2,Sir i think u hav many friends on wiki who can...,can u pls consult any of them?,0
3,I can't find it.,Maybe I didn't manage to guess the right nam...,0
4,"I can't spend too much time, and I'm no specia...","Btw, how is your name pronounced?",0


In [102]:
import json
import random

def get_data(politeness_val, filename):
    sub_df = df[df['politeness']==politeness_val]
    dic = sub_df[['prompt', 'completion']].to_dict(orient='records')
    random.shuffle(dic)
    split_idx = int(len(dic)*0.8)
    train_dic = dic[:split_idx]
    valid_dic = dic[split_idx:]
    with open(filename+'_train.jsonl', 'w') as f:
        for item in train_dic:
            f.write(json.dumps(item) + "\n")
    with open(filename+'_valid.jsonl', 'w') as f:
        for item in valid_dic:
            f.write(json.dumps(item) + "\n")

get_data(1, "polite_dat")
get_data(-1, "impolite_dat")